# Pythonic Data Cleaning With Pandas and NumPy

In [1]:
import pandas as pd
import numpy as np

In [10]:
df = pd.read_csv('https://raw.githubusercontent.com/realpython/python-data-cleaning/master/Datasets/BL-Flickr-Images-Book.csv')

In [6]:
df_towns = pd.read_csv('https://raw.githubusercontent.com/realpython/python-data-cleaning/master/Datasets/university_towns.txt')

df_olympics = pd.read_csv('https://raw.githubusercontent.com/realpython/python-data-cleaning/master/Datasets/olympics.csv')


ParserError: Error tokenizing data. C error: Expected 1 fields in line 8, saw 3


# Dropping Columns in a DataFrame
 

We define a list that contains the names of all the columns we want to drop. Next, we call the drop() function on our object, passing in the inplace parameter as True and the axis parameter as 1. This tells Pandas that we want the changes to be made directly in our object and that it should look for the values to be dropped in the columns of the object.

In [8]:
to_drop = ['Edition Statement',
           'Corporate Author',
           'Corporate Contributors',
           'Former owner',
           'Engraver',
           'Contributors',
           'Issuance type',
           'Shelfmarks']

df.drop(to_drop, inplace=True, axis=1)

In [11]:
df.drop(columns=to_drop, inplace=True)

# Changing the Index of a DataFrame

In [12]:
df['Identifier'].is_unique

True

In [13]:
df = df.set_index('Identifier')

In [14]:
df.loc[206]

Place of Publication                                               London
Date of Publication                                           1879 [1878]
Publisher                                                S. Tinsley & Co.
Title                                   Walter Forbes. [A novel.] By A. A
Author                                                              A. A.
Flickr URL              http://www.flickr.com/photos/britishlibrary/ta...
Name: 206, dtype: object

In [15]:
# Alternative implementation (will not work if index is already changed)
df.set_index('Identifier', inplace=True)


KeyError: "None of ['Identifier'] are in the columns"

In [25]:
df.dtypes.value_counts()

object    6
dtype: int64

# Tidying up Fields in the Data


The regular expression below is meant to find any four digits at the beginning of a string, which suffices for our case. The above is a raw string (meaning that a backslash is no longer an escape character), which is standard practice with regular expressions.

The \d represents any digit, and {4} repeats this rule four times. The ^ character matches the start of a string, and the parentheses denote a capturing group, which signals to Pandas that we want to extract that part of the regex. (We want ^ to avoid cases where [ starts off the string.)

In [21]:
regex = r'^(\d{4})'

In [24]:
 extr = df['Date of Publication'].str.extract(regex, expand=False)
 extr


Identifier
206        1879
216        1868
218        1869
472        1851
480        1857
           ... 
4158088    1838
4158128    1831
4159563     NaN
4159587    1834
4160339    1834
Name: Date of Publication, Length: 8287, dtype: object

In [26]:
df.head()

,Place of Publication,Date of Publication,Publisher,Title,Author,Flickr URL
Identifier,,,,,,
206,London,1879 [1878],S. Tinsley & Co.,Walter Forbes. [A novel.] By A. A,A. A.,http://www.flickr.com/photos/britishlibrary/ta...
216,London; Virtue & Yorston,1868,Virtue & Co.,All for Greed. [A novel. The dedication signed...,"A., A. A.",http://www.flickr.com/photos/britishlibrary/ta...
218,London,1869,"Bradbury, Evans & Co.",Love the Avenger. By the author of “All for Gr...,"A., A. A.",http://www.flickr.com/photos/britishlibrary/ta...
472,London,1851,James Darling,"Welsh Sketches, chiefly ecclesiastical, to the...","A., E. S.",http://www.flickr.com/photos/britishlibrary/ta...
480,London,1857,Wertheim & Macintosh,"[The World in which I live, and my place in it...","A., E. S.",http://www.flickr.com/photos/britishlibrary/ta...


# Cleaning the Entire Dataset Using the applymap Function

In [36]:
uni_towns=[]
with open('university_towns.txt') as f:
    for r in f:
        if '[edit]' in r:
            state = r
        else:
            uni_towns.append((state, r))

In [43]:
towns_df = pd.DataFrame(uni_towns, columns=['state', 'region'])

In [44]:
towns_df

,state,region
0,Alabama[edit]\n,Auburn (Auburn University)[1]\n
1,Alabama[edit]\n,Florence (University of North Alabama)\n
2,Alabama[edit]\n,Jacksonville (Jacksonville State University)[2]\n
3,Alabama[edit]\n,Livingston (University of West Alabama)[2]\n
4,Alabama[edit]\n,Montevallo (University of Montevallo)[2]\n
...,...,...
512,Wisconsin[edit]\n,River Falls (University of Wisconsin–River Fal...
513,Wisconsin[edit]\n,Stevens Point (University of Wisconsin–Stevens...
514,Wisconsin[edit]\n,Waukesha (Carroll University)\n
515,Wisconsin[edit]\n,Whitewater (University of Wisconsin–Whitewater...


In [53]:
def get_state(item):
    if   '[' in item: 
        return item[:item.find('[')]
    elif ' (' in item: 
        return item[:item.find(' (')]
    else: return item 
    

In [54]:
towns_df = towns_df.applymap(get_state)

In [55]:
towns_df

,state,region
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
512,Wisconsin,River Falls
513,Wisconsin,Stevens Point
514,Wisconsin,Waukesha
515,Wisconsin,Whitewater


# Renaming Columns and Skipping Rows

In [58]:
olympics_df = pd.read_csv('olympics.csv', header=1)

In [59]:
olympics_df

,Unnamed: 0,? Summer,01 !,02 !,03 !,Total,? Winter,01 !.1,02 !.1,03 !.1,Total.1,? Games,01 !.2,02 !.2,03 !.2,Combined total
0,Afghanistan (AFG),13,0,0,2,2,0,0,0,0,0,13,0,0,2,2
1,Algeria (ALG),12,5,2,8,15,3,0,0,0,0,15,5,2,8,15
2,Argentina (ARG),23,18,24,28,70,18,0,0,0,0,41,18,24,28,70
3,Armenia (ARM),5,1,2,9,12,6,0,0,0,0,11,1,2,9,12
4,Australasia (ANZ) [ANZ],2,3,4,5,12,0,0,0,0,0,2,3,4,5,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,Independent Olympic Participants (IOP) [IOP],1,0,1,2,3,0,0,0,0,0,1,0,1,2,3
143,Zambia (ZAM) [ZAM],12,0,1,1,2,0,0,0,0,0,12,0,1,1,2
144,Zimbabwe (ZIM) [ZIM],12,3,4,1,8,1,0,0,0,0,13,3,4,1,8
145,Mixed team (ZZX) [ZZX],3,8,5,4,17,0,0,0,0,0,3,8,5,4,17


In [62]:
new_names =  {'Unnamed: 0': 'Country',
              '? Summer': 'Summer Olympics',
              '01 !': 'Gold',
              '02 !': 'Silver',
              '03 !': 'Bronze',
              '? Winter': 'Winter Olympics',
              '01 !.1': 'Gold.1',
              '02 !.1': 'Silver.1',
              '03 !.1': 'Bronze.1',
              '? Games': '# Games',
              '01 !.2': 'Gold.2',
              '02 !.2': 'Silver.2',
              '03 !.2': 'Bronze.2'}

In [65]:
olympics_df.rename(columns=new_names, inplace=True)

# Python Data Cleaning: Recap and Resources
